# Tanzania Tourism Prediction - Prognozy dotyczące turystyki w Tanzanii
<h2>Autorzy:</h2><br>
<ul>
<li>Piotr Janiszek 247678</li>
<li>Kacper Białek 247629</li>
<li>Franciszek Pawlak 247756</li>
<li>Michał Korblit 242427</li>
</ul>

<h3>Imports</h3>

In [ ]:
import numpy as np
import pandas as pd

<h3>Data loadind</h3>


In [ ]:
df_train = pd.read_csv('data/Train.csv')
df_test = pd.read_csv('data/Test.csv')

<h3>Data Cleaning</h3>

In [ ]:
count = df_train.isna().any(axis=1).sum()
df_train.loc[df_train['most_impressing'].isna(), 'most_impressing'] = 'No comments'
count2 = df_train.isna().any(axis=1).sum()

# idzie przez kolumny z nan
for index, row in df_train[df_train.isna().any(axis=1)].iterrows():
    # Completing the travel_with field with the value Alone, when the number of people shows that he travels alone
    if pd.isna(row['travel_with']) & ((row['total_male'] + row['total_female']) == 1):
        df_train.loc[index, 'travel_with'] = 'Alone'
    # Completing the travel_with field with the estimated value when only one gender participated in the trip and the number of people exceeds one
    elif (pd.isna(row['travel_with']) & ((row['total_male'] + row['total_female']) > 1) &
            ((row['total_male'] == 0 | pd.isna(row['total_male'])) | (row['total_female'] == 0 | pd.isna(row['total_female'])))):
        df_train.loc[index, 'travel_with'] = 'Friends/Relatives'

        #df_train.loc[index, 'travel_with'] = 'Children'
        print("Z mordeczkami albo z dzieckiem")
        #print(row['total_male'] + row['total_female'])
    elif pd.isna(row['travel_with']) & ((row['total_male'] + row['total_female']) == 0):
        df_train.loc[index, 'travel_with'] = 'Alone'
        print('predykcja płci')
    elif (row['purpose'] == 'Leisure and Holidays') & (row['total_male'] > 0) & (row['total_female'] > 0) & pd.isna(row['travel_with']):
        print("Wakacje, ale z kim")
    elif pd.isna(row['travel_with']) & (row['total_male'] == 0 | pd.isna(row['total_male'])) | (row['total_female'] == 0 | pd.isna(row['total_female'])):
        print("Brakuje liczby osob czesciowo")
    elif not pd.isna(row['travel_with']):
        print(f"{row['travel_with']}"
              f"\n{row['total_male']}"
              f"\n{row['total_female']}")


count3 = df_train.isna().any(axis=1).sum()

print("Liczba wierszy z brakującymi wartościami:", count)
print("Liczba wierszy z brakującymi wartościami 2:", count2)
print("Liczba wierszy z brakującymi wartościami 3:", count3)

for index, row in df_train[df_train.isna().any(axis=1)].iterrows():
    #print(row)
    pass
